In [2]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks")

# 이미 만들어진 모델 가지고와서 쓰기 
---
# 1. Glove

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
# 변환을 원하는 텍스트

token = Tokenizer()
text = "hello family parents sister"
token.fit_on_texts([text])
word_count = len(token.word_index) # unique한 단어 수

In [5]:
f = open('glove.6B.50d.txt', encoding='utf-8')

word_vectors = {} # 단어의 임베딩 벡터가 들어 있음 (key: 단어/ value: 임베딩 벡터)

for line in f:
  word_vector = line.split()
  word_vectors[word_vector[0]] = word_vector[1:] # word_vector[0] : word
  
print(word_vectors['apple'])

['0.52042', '-0.8314', '0.49961', '1.2893', '0.1151', '0.057521', '-1.3753', '-0.97313', '0.18346', '0.47672', '-0.15112', '0.35532', '0.25912', '-0.77857', '0.52181', '0.47695', '-1.4251', '0.858', '0.59821', '-1.0903', '0.33574', '-0.60891', '0.41742', '0.21569', '-0.07417', '-0.5822', '-0.4502', '0.17253', '0.16448', '-0.38413', '2.3283', '-0.66682', '-0.58181', '0.74389', '0.095015', '-0.47865', '-0.84591', '0.38704', '0.23693', '-1.5523', '0.64802', '-0.16521', '-1.4719', '-0.16224', '0.79857', '0.97391', '0.40027', '-0.21912', '-0.30938', '0.26581']


In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
# 가져온 text 임베딩 처리하기---------------------------------------------------

matrix = np.zeros((word_count, 50)) # 50차원 임베딩 벡터 데이터를 사용했음(glove.6B.50d.txt) 


# 없는 단어에 대한 예외처리가 이 부분에서 필요함

for word, i in token.word_index.items(): 
  matrix[i-1] = np.array(word_vectors[word])

model = Sequential()
model.add(Embedding(word_count, 50, weights=[matrix])) # Embedding Layer
                                                       # weights을 쓰면 embedding vector를 set to be frozen
                                                       # -> not updated during training


Embedding은 단어를 밀집 벡터로 만드는 것이다. 
인공 신경망 용어에서 embedding layer를 만드는 역할을 한다.
Embedding()은 정수 인코딩이 된 단어들을 입력받아서 임베딩을 수행한다.



# 2. Word2Vec

In [12]:
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

matrix = np.zeros((word_count, 300)) 


# 없는 단어에 대한 예외처리가 이 부분에서 필요함

for word, i in token.word_index.items():
  matrix[i] = np.array(word_vectors[word])

# 예시일 뿐 실제 사용가능한 모델은 아님
model = Sequential()
model.add(Embedding(word_count, 50, weights=[matrix])) # Embedding Layer
# 이후 dense layer로 갈 때 flatten layer가 필요(차원맞추기위함)

## - Word2Vec 시각화

- `07_Embedding_Word2Vec.ipynb` 코드 사용

- [Embedding Projector](https://projector.tensorflow.org/)에 .tsv file을 넣어서 임베딩한 단어들을 시각화할 수 있다.

In [7]:
from gensim.models import Word2Vec, KeyedVectors
import csv
import re

def open_csv():
    # csv파일을 연다!
    f = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    doc_list = []

    next(csvreader)
    for f in csvreader:
        line = re.compile("[^\w]").sub(' ', f[0].lower())
        doc_list.append(line.split())

    return doc_list

doc_list = open_csv()
print(doc_list[:3])

model = Word2Vec(sentences=doc_list, size=100, window=3, min_count=3, workers=4, sg=0)

[['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', '1', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'br', 'br', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'unflinching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'timid', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'br', 'br', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'prison', 'whe

In [8]:
# # 모델저장
# model.wv.save_word2vec_format('imdb_w2v')
# # 모델불러오기
# imdb_model = KeyedVectors.load_word2vec_format("imdb_w2v")

In [9]:
# .tsv file로 변경해서 저장
# -> google embedding projector에서 3차원으로(pca로 차원 축소한 것) 단어들의 분포를 확인할 수 있다.
!python -m gensim.scripts.word2vec2tensor --input imdb_w2v --output imdb_w2v

2020-07-29 09:06:32,335 - word2vec2tensor - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/word2vec2tensor.py --input imdb_w2v --output imdb_w2v
2020-07-29 09:06:32,335 - utils_any2vec - INFO - loading projection weights from imdb_w2v
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-07-29 09:06:37,806 - utils_any2vec - INFO - loaded (50964, 100) matrix from imdb_w2v
2020-07-29 09:06:43,730 - word2vec2tensor - INFO - 2D tensor file saved to imdb_w2v_tensor.tsv
2020-07-29 09:06:43,732 - word2vec2tensor - INFO - Tensor metadata file saved to imdb_w2v_metadata.tsv
2020-07-29 09:06:43,744 - word2vec2tensor - INFO - finished running word2vec2tensor.py
